In [6]:
import pandas as pd
import papermill as pm
from datetime import datetime
import logging

In [10]:
import pandas as pd

# Replace 'path_to_your_excel_file.xlsx' with the path to your Excel file
excel_file_1 = '../Master Datasets/raw/hotel_detail_dataset.xlsx'
excel_file_2 = '../Master Datasets/raw/hotel_review.xlsx'

# Read the Excel file
df1 = pd.read_excel(excel_file_1)
df2 = pd.read_excel(excel_file_2)

# Convert to CSV
df1.to_csv('../Master Datasets/raw/hotel_detail_dataset_csv.csv', index=False)
df2.to_csv('../Master Datasets/raw/hotel_review_csv.csv', index=False)


In [17]:

# Execute the update detection notebook

import papermill as pm

pm.execute_notebook(
    'Module01_Hotel_Details_Update_Detection.ipynb',
    'Update_Detection_Output.ipynb',
    parameters={
        'input_path': '../Master Datasets/raw/hotel_detail_dataset_csv.csv',
        'output_path': '../Master Datasets/Hashed and etc/hashed_timestamped_hotel_details.csv'
    }
)

# Depending on the output from the above, decide which notebook to run next
# Assume you have a function to decide if you need to run ontology development
# need_ontology_update = check_for_ontology_update('Update_Detection_Output.ipynb')

# if need_ontology_update:
#     pm.execute_notebook(
#         'Ontology_Development.ipynb',
#         'Ontology_Development_Output.ipynb'
#     )

# # Similar execution can be set for Ontology Population
# pm.execute_notebook(
#     'Ontology_Population.ipynb',
#     'Ontology_Population_Output.ipynb'
# )


Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

{'cells': [{'cell_type': 'code',
   'execution_count': 1,
   'id': 'abc6a885-3727-47a3-8366-c189e76cd5d3',
   'metadata': {'tags': [],
    'papermill': {'exception': False,
     'start_time': '2024-05-26T15:40:54.511842',
     'end_time': '2024-05-26T15:40:54.534322',
     'duration': 0.02248,
     'status': 'completed'},
    'execution': {'iopub.status.busy': '2024-05-26T15:40:54.522918Z',
     'iopub.execute_input': '2024-05-26T15:40:54.523913Z',
     'shell.execute_reply': '2024-05-26T15:40:54.530929Z',
     'iopub.status.idle': '2024-05-26T15:40:54.532273Z'}},
   'outputs': [],
   'source': '# this NoteBook is for update detection for the Hotel Details Dataset using Jupyter Notebooks'},
  {'cell_type': 'code',
   'execution_count': None,
   'id': 'b148418c-fb74-4689-9d75-a2e5736c5fff',
   'metadata': {'tags': [],
    'papermill': {'exception': False,
     'start_time': '2024-05-26T15:40:54.540603',
     'end_time': '2024-05-26T15:40:54.544608',
     'duration': 0.004005,
     'stat

In [9]:
# Setup basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')

# Function to read the High-Water Mark
def get_last_hwm(hwm_path):
    try:
        with open(hwm_path, 'r') as file:
            last_hwm = pd.to_datetime(file.read().strip())
        logging.info(f"Read last HWM: {last_hwm}")
    except FileNotFoundError:
        last_hwm = pd.to_datetime('1900-01-01')  # If no HWM found, use a very early date
        logging.warning("No HWM file found. Setting to default early date.")
    return last_hwm

# Function to load new reviews based on the High-Water Mark
def load_new_reviews(filepath, last_hwm):
    df = pd.read_csv(filepath)
    df['processed_timestamp'] = pd.to_datetime(df['processed_timestamp'])
    new_reviews = df[df['processed_timestamp'] > last_hwm]
    logging.info(f"Loaded {len(new_reviews)} new reviews for processing.")
    return new_reviews

# Function to update the High-Water Mark
def update_hwm(new_hwm, hwm_path):
    with open(hwm_path, 'w') as file:
        file.write(str(new_hwm))
    print(f"High-Water Mark updated to: {new_hwm}")


# Define file paths
input_path = '../Master Datasets/raw/hotel_review_csv.csv'  # Path to the raw master dataset
hwm_path = '../Master Datasets/Hashed and etc/high_water_mark.txt'  # Path to store the High-Water Mark
output_for_preprocessing = '../Master Datasets/Hashed and etc/new_reviews_to_process.csv'  # Where to save new data for preprocessing

# Execution logic in the master notebook
last_hwm = get_last_hwm(hwm_path)  # Corrected variable name
new_reviews = load_new_reviews(input_path, last_hwm)

if not new_reviews.empty:
    new_hwm = new_reviews['processed_timestamp'].max()
     # Ensure datetime conversion if not already done
    new_hwm = pd.to_datetime(new_hwm)
    new_reviews.to_csv(output_for_preprocessing, index=False)
    logging.info("New reviews saved for preprocessing. Triggering preprocessing notebook.")
    # Trigger the preprocessing notebook
    try:
        pm.execute_notebook(
            'Module01_Review_Basic_Preprocessing.ipynb',
            'Module01_Review_Basic_Preprocessing_Output.ipynb',
            parameters={'data_path': output_for_preprocessing}
        )
        update_hwm(new_hwm, hwm_path)
    except Exception as e:
        logging.error("Error during notebook execution: {}".string(e))
else:
    logging.info("No new reviews found to process.")


2024-05-31 11:12:35,268 - INFO: Read last HWM: 2020-05-31 10:31:02.982062
2024-05-31 11:12:36,688 - INFO: Loaded 320836 new reviews for processing.
2024-05-31 11:12:40,712 - INFO: New reviews saved for preprocessing. Triggering preprocessing notebook.
2024-05-31 11:12:40,713 - INFO: Input Notebook:  Module01_Review_Basic_Preprocessing.ipynb
2024-05-31 11:12:40,714 - INFO: Output Notebook: Module01_Review_Basic_Preprocessing_Output.ipynb
2024-05-31 11:12:40,715 - WARNING: Passed unknown parameter: data_path
2024-05-31 11:12:40,717 - WARNING: Input notebook does not contain a cell with tag 'parameters'


Executing:   0%|          | 0/2 [00:00<?, ?cell/s]

2024-05-31 11:12:42,012 - INFO: Executing notebook with kernel: python3


High-Water Mark updated to: 2024-05-31 10:31:02.982062
